<a href="https://colab.research.google.com/github/Ch256981/IMLP347/blob/main/HW2_Ko_I_Hung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BUDT 758J - HW2

## I-Hung Ko

In [1]:
# install PySpark
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# create a SparkSession from pyspark.sql, which is the entry point to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HW2').getOrCreate()

In [3]:
# verify
spark

# Load Dataset

In [5]:
# 703
spark.read.json('hw1_customer.json').createOrReplaceTempView('Customer')
spark.read.json('hw1_order.json').createOrReplaceTempView('Order')
spark.read.json('hw1_orderline.json').createOrReplaceTempView('OrderLine')
spark.read.json('hw1_product.json').createOrReplaceTempView('Product')

In [6]:
# 704
import pandas as pd
pd_customer = pd.read_json('hw1_customer.json')
pd_order = pd.read_json('hw1_order.json')
pd_orderline = pd.read_json('hw1_orderline.json')
pd_product = pd.read_json('hw1_product.json')

In [7]:
# 758J
sp_customer = spark.read.json('hw1_customer.json')
sp_order = spark.read.json('hw1_order.json')
sp_orderline = spark.read.json('hw1_orderline.json')
sp_product = spark.read.json('hw1_product.json')

In [8]:
# Spark SQL
spark.read.json('hw1_customer.json').registerTempTable('customer')
spark.sql('SELECT c.* FROM customer c').show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+-------------+----------+--------------------+-------------+--------------------+-----------+
| customerCity|customerId|        customerName|customerState|      customerStreet|customerZip|
+-------------+----------+--------------------+-------------+--------------------+-----------+
|  Gainesville|         1|Contemporary Casuals|           FL|   1355 S Hines Blvd| 32601-2871|
|        Plano|         2|     Value Furniture|           TX| 15145 S.W. 17th St.| 75094-7743|
|       Albany|         3|    Home Furnishings|           NY|    1900 Allard Ave.| 12209-1125|
|     Carteret|         4|   Eastern Furniture|           NJ|   1925 Beltline Rd.| 07008-3188|
|   Sacramento|         5|         Impressions|           CA|   5585 Westcott Ct.| 94206-4056|
|      Boulder|         6|   Furniture Gallery|           CO|    325 Flatiron Dr.| 80514-4432|
|      Seattle|         7|    Period Furniture|           WA|     394 Rainbow Dr.| 97954-5589|
|  Santa Clara|         8| California Classics|   

# Data Manipulation

#5. How many products (i.e., distinct count) that were ordered (i.e., in each product line) and what was the total ordered quantity in each product line (in the ascending order of product line id)?

In [9]:
# 703
spark.sql('''SELECT p.productLineId, COUNT(DISTINCT p.productId) AS `Count of Products Ordered`, SUM(l.orderedQuantity) AS `Total Ordered Quantity`
       FROM Product p LEFT JOIN Orderline l
       ON p.productId = l.productId
       GROUP BY p.productLineId
       ORDER BY p.productLineId ASC''').show()

+-------------+-------------------------+----------------------+
|productLineId|Count of Products Ordered|Total Ordered Quantity|
+-------------+-------------------------+----------------------+
|            1|                        9|                    29|
|            2|                        3|                     2|
|            3|                        5|                    20|
|            4|                        2|                  NULL|
+-------------+-------------------------+----------------------+



In [10]:
spark.sql('''SELECT p.productLineId, COUNT(DISTINCT l.orderId) AS `Count of Orders`, SUM(l.orderedQuantity) AS `Total Ordered Quantity`
       FROM Product p LEFT JOIN Orderline l
       ON p.productId = l.productId
       GROUP BY p.productLineId
       ORDER BY p.productLineId ASC''').show()

+-------------+---------------+----------------------+
|productLineId|Count of Orders|Total Ordered Quantity|
+-------------+---------------+----------------------+
|            1|              9|                    29|
|            2|              1|                     2|
|            3|              5|                    20|
|            4|              0|                  NULL|
+-------------+---------------+----------------------+



In [11]:
# 704
pd.merge(pd_product, pd_orderline, how='left', on='productId')\
 .groupby('productLineId', as_index=False)\
 .agg({'productId' : 'nunique', 'orderedQuantity': 'sum'})\
 .rename(columns={'productId': 'Count of Product Ordered', 'orderedQuantity': 'Total Ordered Quantity'})

,productLineId,Count of Product Ordered,Total Ordered Quantity
0,1,9,29.0
1,2,3,2.0
2,3,5,20.0
3,4,2,0.0


In [29]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.countDistinct.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sum.html
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import sum
sp_product.join(sp_orderline, 'productId', how='left')\
 .groupBy('productLineId')\
 .agg(countDistinct('productId').alias('Count of Products Ordered'),
    sum('orderedQuantity').alias('Total Ordered Quantity'))\
 .orderBy('productLineId').show()

+-------------+-------------------------+----------------------+
|productLineId|Count of Products Ordered|Total Ordered Quantity|
+-------------+-------------------------+----------------------+
|            1|                        9|                    29|
|            2|                        3|                     2|
|            3|                        5|                    20|
|            4|                        2|                  NULL|
+-------------+-------------------------+----------------------+



#6. What are the id, name (no truncation), finish, and revenue (0 if the product was never been ordered) for all products (in the ascending order of product id)?

In [39]:
# 703
spark.sql('''SELECT p.productId, p.productDescription, p.productFinish, COALESCE (SUM(l.productId * l.orderedQuantity), 0) AS `Revenue`
       FROM Product p LEFT JOIN Orderline l
       ON p.productId = l.productId
       GROUP BY p.productId, p.productDescription, p.productFinish
       ORDER BY p.productId ASC''').show()

+---------+--------------------+-------------+-------+
|productId|  productDescription|productFinish|Revenue|
+---------+--------------------+-------------+-------+
|        1|    Cherry End Table|       Cherry|      5|
|        2| Birch Coffee Tables|        Birch|      8|
|        3|   Oak Computer Desk|          Oak|     33|
|        4|Entertainment Center|       Cherry|     28|
|        5|       Writer's Desk|          Oak|     10|
|        6|    8-Drawer Dresser|        Birch|     12|
|        7|         48 Bookcase|       Walnut|     35|
|        8|         48 Bookcase|          Oak|    120|
|        9|         96 Bookcase|       Walnut|      0|
|       10|         96 Bookcase|          Oak|      0|
|       11|    4-Drawer Dresser|          Oak|      0|
|       12|    8-Drawer Dresser|          Oak|      0|
|       13|          Nightstand|       Cherry|      0|
|       14|       Writer's Desk|        Birch|      0|
|       17|High Back Leather...|      Leather|      0|
|       18

In [40]:
# 704
pd_orderline['Revenue'] = pd_orderline['productId'] * pd_orderline['orderedQuantity']
pd.merge(pd_product, pd_orderline[['productId', 'Revenue']], how='left', on='productId')\
 .groupby(['productId', 'productDescription', 'productFinish'], as_index=False)\
 .agg({'Revenue': 'sum'})\
 .fillna({'Revenue': 0})\
 .astype({'Revenue': int})

,productId,productDescription,productFinish,Revenue
0,1,Cherry End Table,Cherry,5
1,2,Birch Coffee Tables,Birch,8
2,3,Oak Computer Desk,Oak,33
3,4,Entertainment Center,Cherry,28
4,5,Writer's Desk,Oak,10
5,6,8-Drawer Dresser,Birch,12
6,7,48 Bookcase,Walnut,35
7,8,48 Bookcase,Oak,120
8,9,96 Bookcase,Walnut,0
9,10,96 Bookcase,Oak,0


In [59]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.col.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.coalesce.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.lit.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sum.html
from pyspark.sql.functions import col
from pyspark.sql.functions import coalesce
from pyspark.sql.functions import lit
from pyspark.sql.functions import sum
sp_orderline = sp_orderline.withColumn('Revenue', col('productId') * col('orderedQuantity'))
sp_product.join(sp_orderline, 'productId', how='left')\
 .select('productId', 'productDescription', 'productFinish', 'Revenue')\
 .groupBy('productId', 'productDescription', 'productFinish')\
 .agg(coalesce(sum('Revenue'), lit(0)).alias('Revenue'))\
 .orderBy('productId').show()

+---------+--------------------+-------------+-------+
|productId|  productDescription|productFinish|Revenue|
+---------+--------------------+-------------+-------+
|        1|    Cherry End Table|       Cherry|      5|
|        2| Birch Coffee Tables|        Birch|      8|
|        3|   Oak Computer Desk|          Oak|     33|
|        4|Entertainment Center|       Cherry|     28|
|        5|       Writer's Desk|          Oak|     10|
|        6|    8-Drawer Dresser|        Birch|     12|
|        7|         48 Bookcase|       Walnut|     35|
|        8|         48 Bookcase|          Oak|    120|
|        9|         96 Bookcase|       Walnut|      0|
|       10|         96 Bookcase|          Oak|      0|
|       11|    4-Drawer Dresser|          Oak|      0|
|       12|    8-Drawer Dresser|          Oak|      0|
|       13|          Nightstand|       Cherry|      0|
|       14|       Writer's Desk|        Birch|      0|
|       17|High Back Leather...|      Leather|      0|
|       18

#7. What are the id, date, customer name, and overall price for each order (in the ascending order of order id)?

In [ ]:
# 703
spark.sql('''SELECT o.orderId, o.orderDate, c.customerName, SUM(l.orderId * l.orderedQuantity) AS `Overall Price`
       FROM Order o LEFT JOIN Orderline l
       ON o.orderId = l.orderId
       JOIN Customer c
       ON o.customerId = c.customerId
       GROUP BY o.orderId, o.orderDate, c.customerName
       ORDER BY o.orderId ASC''').show()

+-------+----------+--------------------+-------------+
|orderId| orderDate|        customerName|Overall Price|
+-------+----------+--------------------+-------------+
|   1001|2010-10-21|Contemporary Casuals|         5005|
|   1002|2010-10-21| California Classics|         5010|
|   1003|2010-10-22|     Mountain Scenes|         3009|
|   1004|2010-10-22|         Impressions|         4016|
|   1005|2010-10-24|    Home Furnishings|         3015|
|   1006|2010-10-24|     Value Furniture|         5030|
|   1007|2010-10-27|American Euro Lif...|         5035|
|   1008|2010-10-30|Battle Creek Furn...|         6048|
|   1009|2010-11-05|   Eastern Furniture|         5045|
|   1010|2010-11-05|Contemporary Casuals|        10100|
+-------+----------+--------------------+-------------+



In [ ]:
# 704
pd_orderline['Overall Price'] = pd_orderline['orderId'] * pd_orderline['orderedQuantity']
pd.merge(pd.merge(pd_order, pd_customer, how='left', on='customerId'), pd_orderline, how='left', on='orderId')\
 .groupby(['orderId', 'orderDate', 'customerName'], as_index=False)\
 .agg({'Overall Price': 'sum'})\
 .sort_values('orderId')

,orderId,orderDate,customerName,Overall Price
0,1001,2010-10-21,Contemporary Casuals,5005
1,1002,2010-10-21,California Classics,5010
2,1003,2010-10-22,Mountain Scenes,3009
3,1004,2010-10-22,Impressions,4016
4,1005,2010-10-24,Home Furnishings,3015
5,1006,2010-10-24,Value Furniture,5030
6,1007,2010-10-27,American Euro Lifestyles,5035
7,1008,2010-10-30,Battle Creek Furniture,6048
8,1009,2010-11-05,Eastern Furniture,5045
9,1010,2010-11-05,Contemporary Casuals,10100


In [58]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.col.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sum.html
from pyspark.sql.functions import col
from pyspark.sql.functions import sum
sp_orderline = sp_orderline.withColumn('Overall Price', col('orderId') * col('orderedQuantity'))
sp_order.join(sp_customer, 'customerId', 'left')\
 .join(sp_orderline, 'orderId', 'left')\
 .groupBy('orderId', 'orderDate', 'customerName')\
 .agg(sum('Overall Price').alias('Overall Price'))\
 .orderBy('orderId').show()

+-------+----------+--------------------+-------------+
|orderId| orderDate|        customerName|Overall Price|
+-------+----------+--------------------+-------------+
|   1001|2010-10-21|Contemporary Casuals|         5005|
|   1002|2010-10-21| California Classics|         5010|
|   1003|2010-10-22|     Mountain Scenes|         3009|
|   1004|2010-10-22|         Impressions|         4016|
|   1005|2010-10-24|    Home Furnishings|         3015|
|   1006|2010-10-24|     Value Furniture|         5030|
|   1007|2010-10-27|American Euro Lif...|         5035|
|   1008|2010-10-30|Battle Creek Furn...|         6048|
|   1009|2010-11-05|   Eastern Furniture|         5045|
|   1010|2010-11-05|Contemporary Casuals|        10100|
+-------+----------+--------------------+-------------+



#8. What are the id, name (no truncation), address (full address in one column), and count of orders (0 if never ordered) for each customer (in the ascending order of customer id)?

In [56]:
# 703
spark.sql('''SELECT c.customerId, c.customerName, CONCAT(c.customerStreet, ', ',c.customerCity, ', ',c.customerState, ', ',c.customerZip) AS `Full Address`, COALESCE(COUNT(o.orderId), 0) AS `Count of Orders`
       FROM Customer c LEFT JOIN Order o
       ON c.customerId = o.customerId
       GROUP BY c.customerId, c.customerName, c.customerStreet, c.customerCity, c.customerState, c.customerZip
       ORDER BY c.customerId ASC''').show()

+----------+--------------------+--------------------+---------------+
|customerId|        customerName|        Full Address|Count of Orders|
+----------+--------------------+--------------------+---------------+
|         1|Contemporary Casuals|1355 S Hines Blvd...|              2|
|         2|     Value Furniture|15145 S.W. 17th S...|              1|
|         3|    Home Furnishings|1900 Allard Ave.,...|              1|
|         4|   Eastern Furniture|1925 Beltline Rd....|              1|
|         5|         Impressions|5585 Westcott Ct....|              1|
|         6|   Furniture Gallery|325 Flatiron Dr.,...|              0|
|         7|    Period Furniture|394 Rainbow Dr., ...|              0|
|         8| California Classics|816 Peach Rd., Sa...|              1|
|         9|M and H Casual Fu...|3709 First Street...|              0|
|        10|  Seminole Interiors|2400 Rocky Point ...|              0|
|        11|American Euro Lif...|2424 Missouri Ave...|              1|
|     

In [57]:
# 704
pd_customer['Full Address'] = pd_customer['customerStreet'] + ', ' + pd_customer['customerCity'] + ', ' + pd_customer['customerState'] + ', ' + pd_customer['customerZip']
pd.merge(pd_customer, pd_order, how='left', on='customerId')\
 .groupby(['customerId', 'customerName', 'Full Address'], as_index=False)\
 .agg({'orderId': 'count'})\
 .rename(columns={'orderId': 'Count of Orders'})\
 .sort_values('customerId')

,customerId,customerName,Full Address,Count of Orders
0,1,Contemporary Casuals,"1355 S Hines Blvd, Gainesville, FL, 32601-2871",2
1,2,Value Furniture,"15145 S.W. 17th St., Plano, TX, 75094-7743",1
2,3,Home Furnishings,"1900 Allard Ave., Albany, NY, 12209-1125",1
3,4,Eastern Furniture,"1925 Beltline Rd., Carteret, NJ, 07008-3188",1
4,5,Impressions,"5585 Westcott Ct., Sacramento, CA, 94206-4056",1
5,6,Furniture Gallery,"325 Flatiron Dr., Boulder, CO, 80514-4432",0
6,7,Period Furniture,"394 Rainbow Dr., Seattle, WA, 97954-5589",0
7,8,California Classics,"816 Peach Rd., Santa Clara, CA, 96915-7754",1
8,9,M and H Casual Furniture,"3709 First Street, Clearwater, FL, 34620-2314",0
9,10,Seminole Interiors,"2400 Rocky Point Dr., Seminole, FL, 34646-4423",0


In [67]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.concat_ws.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.count.html
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import count
sp_customer = sp_customer.withColumn('Full Address', concat_ws(', ', 'customerStreet', 'customerCity', 'customerState', 'customerZip'))
sp_customer.join(sp_order, 'customerId', 'left')\
 .groupBy('customerId', 'customerName', 'Full Address')\
 .agg(count('orderId').alias('Count of Orders'))\
 .orderBy('customerId').show()

+----------+--------------------+--------------------+---------------+
|customerId|        customerName|        Full Address|Count of Orders|
+----------+--------------------+--------------------+---------------+
|         1|Contemporary Casuals|1355 S Hines Blvd...|              2|
|         2|     Value Furniture|15145 S.W. 17th S...|              1|
|         3|    Home Furnishings|1900 Allard Ave.,...|              1|
|         4|   Eastern Furniture|1925 Beltline Rd....|              1|
|         5|         Impressions|5585 Westcott Ct....|              1|
|         6|   Furniture Gallery|325 Flatiron Dr.,...|              0|
|         7|    Period Furniture|394 Rainbow Dr., ...|              0|
|         8| California Classics|816 Peach Rd., Sa...|              1|
|         9|M and H Casual Fu...|3709 First Street...|              0|
|        10|  Seminole Interiors|2400 Rocky Point ...|              0|
|        11|American Euro Lif...|2424 Missouri Ave...|              1|
|     